In [21]:
import couchdb
import elasticsearch

In [48]:
# Set ES connection
es = elasticsearch.Elasticsearch('http://10.10.100.49/')
es.ping()

True

In [49]:
# Set CouchDB connection
couch = couchdb.Server('http://10.10.100.49/couchdb')

In [29]:
#Recuperando ocorrências
body = """{
   "filter": {
      "and": {
         "filters": [
            {
               "query": {
                  "match": {
                     "family": "ACANTHACEAE"
                  }
               }
            },
            {
               "query": {
                  "match": {
                     "validation.by": "Raquel"
                  }
               }
            }
         ]
      }
   }
}"""

In [30]:
docs = es.search(index='endemicas_rio_de_janeiro', body=body, doc_type='occurrence', size=5000)
print "Got %d documents" % len(docs['hits']['hits'])

Got 121 documents


In [33]:
not_found = []
diff_len = []

for doc in docs['hits']['hits']:
    # Get alias or index name
    #aliases = es.indices.get_aliases(index=doc["_index"])
    #if len(aliases[doc["_index"]]["aliases"]) == 0:
    #    doc_index = doc["_index"]
    #else:
    #    doc_index = aliases[doc["_index"]]["aliases"].keys()[0]
    doc_index = 'endemicas_rio_de_janeiro'
    doc_id = doc['_id']
    
    # Get document
    try:
        db = couch[doc_index]
    except couchdb.ServerError:
        print doc_index
        break
    doc_couch = db.get(doc_id)
    if doc_couch == None:
        not_found.append(doc_id)
        continue
        
    # Get fields
    doc_couch['validation']['by'] = 'Sheila Profice'
    contributor = doc_couch['metadata']['contributor']
    contact = doc_couch['metadata']['contact']
    contributor_array = contributor.split(';')
    contact_array = contact.split(';')
    contributor_array = [unicode(person.strip()) for person in contributor_array]
    contact_array = [unicode(person.strip()) for person in contact_array]
    if len(contributor_array) != len(contact_array):
        diff_len.append(doc_id)
        continue
    contributor_array.append('Sheila Profice')
    contact_array.append('sprofice@jbrj.gov.br')
    doc_couch['metadata']['contributor'] = " ; ".join(contributor_array)
    doc_couch['metadata']['contact'] = " ; ".join(contact_array)
    
    db.update([doc_couch])

In [32]:
print "Not found: %d" % len(not_found)
print "Diff len: %d" % len(diff_len)

Not found: 0
Diff len: 0


In [42]:
#Recuperando avaliações
body = """{
    "filter": {"exists": {
       "field": "validations.comment"
    }},
    "query": {"match": {
       "taxon.family": "ACANTHACEAE"
    }}
}"""

In [50]:
docs = es.search(index='endemicas_rio_de_janeiro', body=body, doc_type='profile', size=5000)
print "Got %d documents" % len(docs['hits']['hits'])

Got 9 documents


In [53]:
for doc in docs['hits']['hits']:
    doc_index = 'endemicas_rio_de_janeiro'
    doc_id = doc['_id']
    
    # Get document
    try:
        db = couch[doc_index]
    except couchdb.ServerError:
        print doc_index
        break
    doc_couch = db.get(doc_id)
    if doc_couch == None:
        not_found.append(doc_id)
        continue
        
    validations = doc_couch['validations']
    print doc_id
    for ix, val in enumerate(validations):
        print val['metadata']['creator'], val['metadata']['contact']
        doc_couch['validations'][ix]['metadata']['creator'] = 'Sheila Profice'
        doc_couch['validations'][ix]['metadata']['contact'] = 'sprofice@jbrj.gov.br'
        contributor = doc_couch['metadata']['contributor']
        contact = doc_couch['metadata']['contact']
        contributor_array = contributor.split(';')
        contact_array = contact.split(';')
        contributor_array = [unicode(person.strip()) for person in contributor_array]
        contact_array = [unicode(person.strip()) for person in contact_array]
        contributor_array.append('Sheila Profice')
        contact_array.append('sprofice@jbrj.gov.br')
    print doc_couch['metadata']['contributor']
    print doc_couch['metadata']['contact']
    doc_couch['metadata']['contributor'] = " ; ".join(contributor_array)
    doc_couch['metadata']['contact'] = " ; ".join(contact_array)
    print "\n"
    
    #db.update([doc_couch])

profile:550971e392515
Sheila Profice sprofice@jbrj.gov.br
Raquel Negrão ; Sheila Profice
raquel@cncflora.net ; sprofice@jbrj.gov.br


profile:55102646b8d60
Sheila Profice sprofice@jbrj.gov.br
Raquel Negrão ; Sheila Profice
raquel@cncflora.net ; sprofice@jbrj.gov.br


profile:5509811034926
Sheila Profice sprofice@jbrj.gov.br
Sheila Profice sprofice@jbrj.gov.br
Raquel Negrão ; Sheila Profice
raquel@cncflora.net ; sprofice@jbrj.gov.br


profile:5509a8029b6e8
Sheila Profice sprofice@jbrj.gov.br
Sheila Profice sprofice@jbrj.gov.br
Sheila Profice sprofice@jbrj.gov.br
Sheila Profice sprofice@jbrj.gov.br
Raquel Negrão ; Sheila Profice
raquel@cncflora.net ; sprofice@jbrj.gov.br


profile:55019eb345576
Sheila Profice sprofice@jbrj.gov.br
Sheila Profice sprofice@jbrj.gov.br
Raquel Negrão ; Sheila Profice
raquel@cncflora.net ; sprofice@jbrj.gov.br


profile:55100a2c8c7e0
Sheila Profice sprofice@jbrj.gov.br
Sheila Profice sprofice@jbrj.gov.br
Raquel Negrão ; Sheila Profice
raquel@cncflora.net ; spr